![ga4](https://www.google-analytics.com/collect?v=2&tid=G-6VDTYWLKX6&cid=1&en=page_view&sid=1&dl=statmike%2Fvertex-ai-mlops%2FDev%2Fnew&dt=BigQuery+-+Columns.ipynb)

# BigQuery | Columns

Explaining BigQuery Column types by examples!